In [1]:
import os
import pandas as pd
import boto3
from io import BytesIO


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ellaquan/Project/organized_metadata.xlsx'

In [18]:
# List all objects in the S3 bucket to confirm the path
response = s3.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    for obj in response['Contents']:
        print("Found file:", obj['Key'])
else:
    print("No objects found in bucket:", bucket_name)


Found file: CMMD-D2/D2-0001_1-1.dcm
Found file: CMMD-D2/D2-0001_1-2.dcm
Found file: CMMD-D2/D2-0001_1-3.dcm
Found file: CMMD-D2/D2-0001_1-4.dcm
Found file: CMMD-D2/D2-0002_1-1.dcm
Found file: CMMD-D2/D2-0002_1-2.dcm
Found file: CMMD-D2/D2-0002_1-3.dcm
Found file: CMMD-D2/D2-0002_1-4.dcm
Found file: CMMD-D2/D2-0003_1-1.dcm
Found file: CMMD-D2/D2-0003_1-2.dcm
Found file: CMMD-D2/D2-0003_1-3.dcm
Found file: CMMD-D2/D2-0003_1-4.dcm
Found file: CMMD-D2/D2-0004_1-1.dcm
Found file: CMMD-D2/D2-0004_1-2.dcm
Found file: CMMD-D2/D2-0004_1-3.dcm
Found file: CMMD-D2/D2-0004_1-4.dcm
Found file: CMMD-D2/D2-0005_1-1.dcm
Found file: CMMD-D2/D2-0005_1-2.dcm
Found file: CMMD-D2/D2-0005_1-3.dcm
Found file: CMMD-D2/D2-0005_1-4.dcm
Found file: CMMD-D2/D2-0006_1-1.dcm
Found file: CMMD-D2/D2-0006_1-2.dcm
Found file: CMMD-D2/D2-0006_1-3.dcm
Found file: CMMD-D2/D2-0006_1-4.dcm
Found file: CMMD-D2/D2-0007_1-1.dcm
Found file: CMMD-D2/D2-0007_1-2.dcm
Found file: CMMD-D2/D2-0007_1-3.dcm
Found file: CMMD-D2/D2-0007_

In [17]:

# Initialize S3 client
s3 = boto3.client('s3')
bucket_name = 'ella-dlbiomarkers'
clinical_data_key = 'D2_Clinical.xlsx'  # Updated key for the clinical data in S3
s3_dicom_prefix = 'CMMD-D2/'  # Directory path within the S3 bucket for DICOM files

# Load clinical data directly from S3
obj = s3.get_object(Bucket=bucket_name, Key=clinical_data_key)
clinical_data = pd.read_excel(BytesIO(obj['Body'].read()))


NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [13]:

# Check clinical data structure
print("Clinical Data Columns:", clinical_data.columns)

Clinical Data Columns: Index(['ID1', 'LeftRight', 'Age', 'number', 'abnormality', 'classification',
       'subtype'],
      dtype='object')


In [14]:
# Define view and laterality mapping
view_map = {
    '1-1': ('L', 'CC'),
    '1-2': ('L', 'MLO'),
    '1-3': ('R', 'CC'),
    '1-4': ('R', 'MLO')
}

# Create a list to store the new metadata
organized_data_with_clinical = []


In [16]:
# List all DICOM files in the S3 directory
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_dicom_path)

# Check if files were found
if 'Contents' in response:
    # Iterate over files in the S3 directory
    for obj in response['Contents']:
        file_key = obj['Key']
        file_name = file_key.split('/')[-1]
        
        if file_name.endswith(".dcm"):
            # Extract the ID and view information from the file name
            parts = file_name.split('_')
            patient_id = parts[0]  # "D2-0001"
            exam_id = parts[1].replace('.dcm', '')  # "1-1", "1-2", etc.

            # Get the laterality and view based on exam_id
            laterality, view = view_map.get(exam_id, ('Unknown', 'Unknown'))

            # Match with the clinical data
            match = clinical_data[clinical_data['ID1'] == patient_id]

            if not match.empty:
                # Add the relevant clinical data columns to the metadata
                for _, row in match.iterrows():
                    organized_data_with_clinical.append({
                        'patient_id': patient_id,
                        'exam_id': exam_id,
                        'laterality': laterality,
                        'view': view,
                        'file_path': f"s3://{bucket_name}/{file_key}",  # S3 path to the file
                        'years_to_cancer': 0,
                        'years_to_last_followup': 10,
                        'split_group': 'train',
                        'Age': row['Age'],
                        'number': row['number'],
                        'abnormality': row['abnormality'],
                        'classification': row['classification'],
                        'subtype': row['subtype'],
                        'target': row['target']
                    })
else:
    print("No files found in S3 directory:", s3_dicom_prefix)

# Print collected data for verification
print("Collected Data Sample:", organized_data_with_clinical[:5])

# Convert to DataFrame
organized_df_with_clinical = pd.DataFrame(organized_data_with_clinical)

# Save to a local Excel file for debugging
organized_df_with_clinical.to_excel("test_output.xlsx", index=False)

# Check if DataFrame has data before saving to S3
print("DataFrame Shape:", organized_df_with_clinical.shape)

# Define the output path in S3
output_s3_key = 'output/organized_metadata_with_clinical.xlsx'

# Save the organized data to S3 if DataFrame is not empty
if not organized_df_with_clinical.empty:
    with BytesIO() as output_buffer:
        organized_df_with_clinical.to_excel(output_buffer, index=False)
        output_buffer.seek(0)
        s3.put_object(Bucket=bucket_name, Key=output_s3_key, Body=output_buffer)
    print(f"Organized metadata with clinical data saved to s3://{bucket_name}/{output_s3_key}")
else:
    print("No data to save to S3.")

No files found in S3 directory: CMMD-D2/
Collected Data Sample: []
DataFrame Shape: (0, 0)
No data to save to S3.


In [ ]:

# Save the organized data to S3
with BytesIO() as output_buffer:
    organized_df_with_clinical.to_excel(output_buffer, index=False)
    output_buffer.seek(0)
    s3.put_object(Bucket=bucket_name, Key=output_s3_key, Body=output_buffer)

print(f"Organized metadata with clinical data saved to s3://{bucket_name}/{output_s3_key}")